In [1]:
import os
import cv2

In [2]:
RAW_VIDEO = 'Data/Raw/Videos'
RAW_IMAGES = 'Data/Raw/Images'

# VIDEO TO IMAGES

In [7]:
def convertVideoToImages(video_name, class_label, video_idx, step=10, output_path=RAW_IMAGES):
    print("Converting ", video_name, " Class: ", class_label)
    # Get output path and make output image folder
    output_folder = os.path.join(output_path, class_label)
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
    
    frame_count = 0
    name_count = 0

    # Convert video to images
    vidcap = cv2.VideoCapture(video_name)
    success,image = vidcap.read()
    while success:
        if frame_count % step == 0:
            name_count += 1
            img_name = 'image{}_{}.jpg'.format(video_idx, name_count)
            img_path = os.path.join(output_folder, img_name)
            image = cv2.rotate(image, cv2.ROTATE_180)
            cv2.imwrite(img_path, image)
        
        success,image = vidcap.read()
        frame_count += 1
    
    print("Saved {} images to folder {}\n".format(name_count, class_label))

In [8]:
class_labels = os.listdir(RAW_VIDEO)

for class_label in class_labels:
    if not class_label == '.DS_Store':
        video_path = os.path.join(RAW_VIDEO, class_label)
        videos = os.listdir(video_path)
        for idx, video_name in enumerate(videos):
            if not video_name == '.DS_Store':
                convertVideoToImages(os.path.join(video_path, video_name), class_label, idx, step=5)

Converting  Data/Raw/Videos/push_down/IMG_5649.MOV  Class:  push_down
Saved 211 images to folder push_down

Converting  Data/Raw/Videos/push_down/push_down.mov  Class:  push_down
Saved 66 images to folder push_down

Converting  Data/Raw/Videos/push_down/IMG_5652.MOV  Class:  push_down
Saved 365 images to folder push_down

Converting  Data/Raw/Videos/push_down/IMG_5650.MOV  Class:  push_down
Saved 192 images to folder push_down

Converting  Data/Raw/Videos/push_down/IMG_5651.MOV  Class:  push_down
Saved 186 images to folder push_down

Converting  Data/Raw/Videos/squat_down/IMG_5646.MOV  Class:  squat_down
Saved 358 images to folder squat_down

Converting  Data/Raw/Videos/squat_down/IMG_5644.MOV  Class:  squat_down
Saved 202 images to folder squat_down

Converting  Data/Raw/Videos/squat_down/IMG_5645.MOV  Class:  squat_down
Saved 522 images to folder squat_down

Converting  Data/Raw/Videos/push_up/IMG_5647.MOV  Class:  push_up
Saved 721 images to folder push_up

Converting  Data/Raw/Vide

# IMAGES TO POSE DATA

In [11]:
RAW_IMAGES = 'Data/Raw/Images'
PROCESSED_IMAGES = 'Data/Processed/Images'
CSV_OUTPUT = 'Data/Processed/training_output_csv.csv'

In [13]:
import csv
import cv2
import numpy as np
import os
import sys
import tqdm

from mediapipe.python.solutions import drawing_utils as mp_drawing
from mediapipe.python.solutions import pose as mp_pose


with open(CSV_OUTPUT, 'w') as csv_out_file:
  csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)

  # Folder names are used as pose class names.
  pose_class_names = sorted([n for n in os.listdir(RAW_IMAGES) if not n.startswith('.')])

  for pose_class_name in pose_class_names:
    print('Bootstrapping ', pose_class_name, file=sys.stderr)

    if not os.path.exists(os.path.join(PROCESSED_IMAGES, pose_class_name)):
      os.makedirs(os.path.join(PROCESSED_IMAGES, pose_class_name))

    image_names = sorted([
        n for n in os.listdir(os.path.join(RAW_IMAGES, pose_class_name))
        if not n.startswith('.')])

    for image_name in tqdm.tqdm(image_names, position=0):
      # Load image.
      input_frame = cv2.imread(os.path.join(RAW_IMAGES, pose_class_name, image_name))
      input_frame = cv2.cvtColor(input_frame, cv2.COLOR_BGR2RGB)

      # Initialize fresh pose tracker and run it.
      with mp_pose.Pose() as pose_tracker:
        result = pose_tracker.process(image=input_frame)
        pose_landmarks = result.pose_landmarks
      
      # Save image with pose prediction (if pose was detected).
      output_frame = input_frame.copy()
      if pose_landmarks is not None:
        
        mp_drawing.draw_landmarks(
            image=output_frame,
            landmark_list=pose_landmarks,
            connections=mp_pose.POSE_CONNECTIONS)

      output_frame = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
      cv2.imwrite(os.path.join(PROCESSED_IMAGES, pose_class_name, image_name), output_frame)
      
      # Save landmarks.
      if pose_landmarks is not None:
        # Check the number of landmarks and take pose landmarks.
        assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
        pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]

        # Map pose landmarks from [0, 1] range to absolute coordinates to get
        # correct aspect ratio.
        frame_height, frame_width = output_frame.shape[:2]
        pose_landmarks *= np.array([frame_width, frame_height, frame_width])

        # Write pose sample to CSV.
        pose_landmarks = np.around(pose_landmarks, 5).flatten().astype('str').tolist()
        csv_out_writer.writerow([image_name, pose_class_name] + pose_landmarks)


Bootstrapping  push_down
100%|██████████| 1020/1020 [02:18<00:00,  7.37it/s]
Bootstrapping  push_up
100%|██████████| 721/721 [01:40<00:00,  7.15it/s]
